In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pandas as pd
import xarray as xr
import intake

import odiac_ff

import util

In [22]:
cluster, client = util.get_ClusterClient()
cluster.adapt(minimum_jobs=2, maximum_jobs=5)
cluster

In [23]:
date_range = pd.date_range(start='2000-01-01', end='2020-12-31', freq='1M')
ds_list_list = client.map(odiac_ff.retrieve_datasets, date_range)
ds_list_list = client.gather(ds_list_list)
ds_list = [ds for ds_list in ds_list_list for ds in ds_list]

ds = xr.concat(ds_list, dim='time', compat='override', coords='minimal').squeeze()
ds

<xarray.Dataset>
Dimensions:   (lon: 360, time: 184104, lat: 180)
Coordinates:
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * time      (time) object 2000-01-01 00:30:00 ... 2020-12-31 23:30:00
Data variables:
    emission  (time, lat, lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
    area      (time, lat, lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
Attributes:
    total_emis:         18.36 TgC
    odiac_total:  Original ODIAC emission for this month =       19.03 TgC/day

In [ ]:
ds